In [3]:
from pyntcloud import PyntCloud 
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt 
import pandas as pd
import os
import sys
import pdb
from pcdiff import knn_graph
import jax
import jax.numpy as jnp
from jax import jit
from numpy import linalg as LA
import torch

In [3]:
def L2norm_nbh(data, comparison_size, origin_index=0):
    # Convert PyTorch tensor to JAX array if needed
    if isinstance(data, torch.Tensor):
        data = jax.device_put(data.detach().cpu().numpy())  # Convert to JAX arrayee

    data_jax = jnp.array(data)[:, 1:comparison_size+1, :]  # Ensure JAX array

    # Select the origin point from each batch (shape: (batch_size, num_features))
    origin = jnp.array(data)[:, origin_index, :]

    # Compute L2 norm for each row in the neighborhood (shape: (batch_size, num_neighbors))
    dist = jnp.linalg.norm(data_jax - origin[:, None, :], axis=2)

    return dist  # Shape: (batch_size, num_neighbors)

In [4]:
def Edge_and_Plane(path, edge_k = 10, plane_overlap = 6, edge_thresh = 0.06, plane_thresh = 0.001, plane_deviation = 0.0001, min_planesize = 20):
    # Load the XYZ file as a DataFrame
    df = pd.read_csv(path, sep=" ", usecols=[0, 1, 2], names=["x", "y", "z"])
    df["row_index"] = df.index
    # Convert DataFrame to a PyntCloud object
    pc = PyntCloud(df)
    x = pc.points['x'].values.reshape(-1,1)
    y = pc.points['y'].values.reshape(-1,1)
    z = pc.points['z'].values.reshape(-1,1)
    pc_array = np.hstack((x,y,z))

    tree = pc.add_structure("kdtree")
    nbh_curv = pc.get_neighbors(k=edge_k, kdtree=tree) 
    eigen_curv = pc.add_scalar_field("eigen_values", k_neighbors=nbh_curv)
    curvfield = pc.add_scalar_field("curvature", ev = eigen_curv)
    curvature = pc.points['curvature('+str(edge_k+1)+')'].values

    tree = pc.add_structure("kdtree")
    nbh_omni = pc.get_neighbors(k=min_planesize, kdtree=tree) 
    eigen_omni = pc.add_scalar_field("eigen_values", k_neighbors=nbh_omni)
    omnivarfield = pc.add_scalar_field("omnivariance", ev = eigen_omni)
    omnivaraiance = pc.points['omnivariance('+str(min_planesize+1)+')'].values

    nbh_origin = np.hstack((pc.points['row_index'].values.reshape(-1,1), nbh_curv))
    
    plane_deviation = np.mean(L2norm_nbh(pc_array[nbh_origin,:],5)) * plane_deviation

    plane_size = min_planesize

    while plane_size >= min_planesize:
        plane_field = pc.add_scalar_field("plane_fit", max_dist=plane_deviation, max_iterations=500)
        plane = pc.points['is_plane'].values.reshape(-1,1)
        plane_size = np.sum(plane[:,0], axis=0)
        
        if plane_size >= min_planesize:
            row_indicies = pc.points['row_index'].values.reshape(-1,1)
            plane_col = np.zeros_like(x)
            plane_col[row_indicies[np.where(plane == 1)[0]]] = 1
            pc_array = np.hstack((pc_array,plane_col))
            pc.points = pc.points[pc.points['is_plane'] != 1]

    if pc_array.shape[1] == 3:
        pc_array = np.hstack((pc_array, np.zeros_like(x).reshape(-1,1)))

    num_planes = len(pc_array[0,3:])
    plane_count = np.count_nonzero(np.sum(pc_array[nbh_origin[:,:plane_overlap],3:], axis=1), axis=1)
    edge_index = np.where((plane_count > 1) | (curvature >= edge_thresh))[0]
    edges = np.zeros_like(x)
    edges[edge_index,0] = 1

    plane_index = np.where(((plane_count == 1) | (omnivaraiance <= plane_thresh)) & (edges[:,0] != 1))[0]
    planes = np.zeros_like(x)
    planes[plane_index,0] = 1
    return edges, planes

In [5]:
def Scalar_fields(path, k = 50):
    df = pd.read_csv(path, sep=" ", usecols=[0, 1, 2], names=["x", "y", "z"])
    df["row_index"] = df.index
    pc = PyntCloud(df)
    x = pc.points['x'].values.reshape(-1,1)
    y = pc.points['y'].values.reshape(-1,1)
    z = pc.points['z'].values.reshape(-1,1)
    pc_array = np.hstack((x,y,z))

    tree = pc.add_structure("kdtree")
    nbh_curv = pc.get_neighbors(k=k, kdtree=tree)

    eigenField = pc.add_scalar_field("eigen_values", k_neighbors=nbh_curv)

    curvfield = pc.add_scalar_field("curvature", ev = eigenField)
    curvature = pc.points['curvature('+str(k+1)+')'].values.reshape(-1,1)

    linfield = pc.add_scalar_field("linearity", ev = eigenField)
    linearity = pc.points['linearity('+str(k+1)+')'].values.reshape(-1,1)

    planfield = pc.add_scalar_field("planarity", ev = eigenField)
    planarity = pc.points['planarity('+str(k+1)+')'].values.reshape(-1,1)

    spherefield = pc.add_scalar_field("sphericity", ev = eigenField)
    sphericity = pc.points['sphericity('+str(k+1)+')'].values.reshape(-1,1)

    omnivarfield = pc.add_scalar_field("omnivariance", ev = eigenField)
    omnivaraiance = pc.points['omnivariance('+str(k+1)+')'].values.reshape(-1,1)

    eigentropyfield = pc.add_scalar_field("eigenentropy", ev = eigenField)
    eigentropy = pc.points['eigenentropy('+str(k+1)+')'].values.reshape(-1,1)

    anisofield = pc.add_scalar_field("anisotropy", ev = eigenField)
    anisotropy = pc.points['anisotropy('+str(k+1)+')'].values.reshape(-1,1)

    eigensum_field = pc.add_scalar_field("eigen_sum", ev = eigenField)
    eigensum = pc.points['eigen_sum('+str(k+1)+')'].values.reshape(-1,1)

    return curvature, linearity, planarity, sphericity, omnivaraiance, eigentropy, anisotropy, eigensum


In [6]:
def Get_variables(path, k=50, edge_k=10, edge_thresh=0.06, plane_thresh=0.001, plane_overlap=6, min_planesize=20, plot="No", save="yes"):
    curvature, linearity, planarity, sphericity, omnivaraiance, eigentropy, anisotropy, eigensum = Scalar_fields(path, k=k)
    edge, plane = Edge_and_Plane(path, edge_k=edge_k, plane_overlap=plane_overlap, edge_thresh=edge_thresh, plane_thresh=plane_thresh, min_planesize=min_planesize)
    xyz = np.loadtxt(path)[:,0:3]

    PC_variables = np.hstack((xyz, edge, plane, curvature, linearity, planarity, sphericity, omnivaraiance, eigentropy, anisotropy, eigensum))

    if save == "yes":
        np.savetxt("./detected_edge/PC_variables.xyz", PC_variables, fmt="%.6f", delimiter=" ")

    if plot == "yes":
        # edge
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=edge, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='edge')

        plt.show()

        # plane
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=plane, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Plane')

        plt.show()

        # curvature
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=curvature, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Curvature')

        plt.show()

        #Linearity
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=linearity, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Linearity')

        plt.show()

        #Planarity
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        xyz = np.loadtxt(path)

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=planarity, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Planarity')

        plt.show()

        #Sphericity
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        xyz = np.loadtxt(path)

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=sphericity, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Sphericity')

        plt.show()

        #Omnivariance
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        xyz = np.loadtxt(path)

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=omnivaraiance, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Omnivariance')

        plt.show()

        #Eigentropy
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        xyz = np.loadtxt(path)

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=eigentropy, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Eigentropy')

        plt.show()

        #Anisotropy
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        xyz = np.loadtxt(path)

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=anisotropy, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Anisotropy')

        plt.show()

        #Eigensum
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        xyz = np.loadtxt(path)

        scatter = ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2],
                            c=eigensum, cmap='viridis')

        fig.colorbar(scatter, ax=ax, label='Eigensum')

        plt.show()
    
    return PC_variables


In [11]:
path = "C:/Users/aagaa/OneDrive - Aarhus universitet/Dokumenter/GitHub/R-D/Code/Leihui Code/dataset/SelfGeneratedClouds/bevel_gear.xyz"
xyz = np.loadtxt(path)

edge_index = knn_graph(xyz, 20)

pos, col = edge_index
print(col[:21])

[    0 11491  2754 33117 34370 28185  8182  8181  2534 37663 36103 11727
 29037  8183    85  2755  2690 49349 11803 33986     1]


In [7]:
path = "C:/Users/aagaa/OneDrive - Aarhus universitet/Dokumenter/GitHub/R-D/Drawings/STL/test.xyz"
path = "C:/Users/aagaa/OneDrive - Aarhus universitet/Dokumenter/GitHub/R-D/Code/Leihui Code/dataset/SelfGeneratedClouds/bevel_gear.xyz"
path = "C:/Users/aagaa/OneDrive - Aarhus universitet/Dokumenter/GitHub/R-D/Code/Leihui Code/dataset/scanning_repository/bunny.xyz"
Get_variables(path, k=20,plot="no", edge_k=10,edge_thresh=0.06)
#Scalar_fields(path)
#Edge_and_Plane(path)

array([[-3.78297000e-02,  1.27940000e-01,  4.47467000e-03, ...,
         6.44228227e-05,  9.99132101e-01,  5.00865471e-06],
       [-4.47794000e-02,  1.28887000e-01,  1.90497000e-03, ...,
         5.75139141e-05,  9.79426799e-01,  4.39773722e-06],
       [-6.80095000e-02,  1.51244000e-01,  3.71953000e-02, ...,
         7.48638609e-05,  9.92956345e-01,  5.87040427e-06],
       ...,
       [-7.04544000e-02,  1.50585000e-01, -4.34585000e-02, ...,
         4.32208281e-05,  8.65446235e-01,  3.19072411e-06],
       [-3.10262000e-02,  1.53727990e-01, -3.54608000e-03, ...,
         5.43097223e-05,  9.72562391e-01,  4.13532225e-06],
       [-4.00442000e-02,  1.53620000e-01, -8.16685000e-03, ...,
         6.21268611e-05,  9.96861474e-01,  4.80262087e-06]],
      shape=(35947, 13))